In [0]:
# %pip install databricks-vectorsearch
# %pip install --upgrade databricks-langchain langchain-community langchain databricks-sql-connector
# dbutils.library.restartPython()
# %pip install -U mlflow
# dbutils.library.restartPython()
# %pip install databricks-sql-connector pandas
# dbutils.library.restartPython()


import mlflow
import os
from openai import OpenAI
from databricks.vector_search.client import VectorSearchClient


In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
# w.secrets.create_scope(
#     scope="pizza-secrets"
# )
# Now you can add your secret as before
w.secrets.put_secret(
    scope="pizza-secrets",
    key="DATABRICKS_TOKEN",
    string_value=dbutils.notebook.entry_point
        .getDbutils()
        .notebook()
        .getContext()
        .apiToken()
        .get()
)

In [0]:
import os
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()

os.environ["DATABRICKS_HOST"] = ctx.apiUrl().get()
os.environ["DATABRICKS_TOKEN"] = ctx.apiToken().get()

In [0]:
# from databricks import sql
# import pandas as pd
# import os


# def run_sql_warehouse_query(query: str) -> pd.DataFrame:
#     """
#     Executes a SQL query against Databricks SQL Warehouse
#     and returns a Pandas DataFrame.
#     """

#     with sql.connect(
#         server_hostname="dbc-741c7540-6155.cloud.databricks.com",
#         http_path="/sql/1.0/warehouses/e2eb5bcf69add002",
#         access_token=os.environ["DATABRICKS_TOKEN"],
#     ) as connection:
#         return pd.read_sql(query, connection)
    
# df = run_sql_warehouse_query("select * from retail_analytics.pizza.gold_pizza_metrics_daily")

# # Convert for MLflow output safety
# result_json = df.to_json(orient="records")



In [0]:
# import pandas as pd
# import mlflow.pyfunc
# import os
# from openai import OpenAI
# from databricks_langchain import DatabricksVectorSearch
# from databricks import sql


# class PizzaRAGPyFuncModel(mlflow.pyfunc.PythonModel):

#     # -----------------------------
#     # LOAD CONTEXT
#     # -----------------------------
#     def load_context(self, context):

#         self.client = OpenAI(
#             base_url=f"{os.environ['DATABRICKS_HOST']}/serving-endpoints",
#             api_key=os.environ["DATABRICKS_TOKEN"],
#         )

#         # Vector Search (ONLY semantic tables)
#         self.ingredient_summary_vs = DatabricksVectorSearch(
#             index_name="retail_analytics.pizza.gold_semantic_ingredient_summary_index",
#             endpoint="pizza-rag-endpoint",
#         )

#         self.pizza_vs = DatabricksVectorSearch(
#             index_name="retail_analytics.pizza.gold_semantic_pizza_index",
#             endpoint="pizza-rag-endpoint",
#         )

#     # -----------------------------
#     # INTENT DETECTION
#     # -----------------------------
#     def detect_query_intent(self, question: str) -> str:
#         q = question.lower()

#         if any(k in q for k in [
#             "how many", "how much", "total", "sum","analyse"
#             "count", "sold", "revenue", "income",
#             "most", "highest", "top", "maximum",
#             "month", "year", "january", "february",
#             "march", "april", "may", "june", "july",
#             "august", "september", "october", "november", "december","sales"
#         ]):
#             return "SQL_ANALYTICS"

#         if any(k in q for k in [
#             "ingredient", "ingredients", "what is used",
#             "what does", "describe", "explain"
#         ]):
#             return "SEMANTIC"

#         return "SEMANTIC"

#     # -----------------------------
#     # SQL GENERATION (CONTROLLED)
#     # -----------------------------
#     def generate_sql(self, question: str) -> str:
#         """
#         LLM-assisted SQL generation over semantic FACT tables
#         """
#         prompt = f"""
# You are a senior data analyst.

# You are an expert Databricks Spark SQL engineer generate a Spark SQL query ONLY using the tables below:

# 1. retail_analytics.pizza.gold_pizza_metrics_monthly
#    columns: pizza_name_id, month, year, total_pizzas_sold, total_revenue
# 2. retail_analytics.pizza.gold_ingredient_usage
#    columns: pizza_name_id, ingredient, month, year, total_pizza_month,items_qty_in_grams,total_ingredient_grams
# 3. retail_analytics.pizza.gold_pizza_metrics_daily
#    columns: pizza_name_id, total_pizza_day,day,month, year,revenue_perday
# Generate ONLY valid Databricks Spark SQL.
# Strict rules:
# - The SQL engine is Databricks (Apache Spark SQL), NOT SQL Server, Postgres, or MySQL.
# - Never reuse column aliases in the same SELECT.
# - if question is on specific date use daily table if it is for year or month use monthly table and use both table if you revenue is needed to be calulated
# - if question is on ingredient use ingredient table and if needed for some analysis join all three tables for results
# - Use only the provided schema
# - Never apply aggregate functions (SUM, AVG, COUNT) to an alias.
# - Always aggregate raw columns first.
# - All non-aggregated columns must appear in GROUP BY.
# - Do NOT use SELECT aliases in ORDER BY if they are derived from aggregates.
# - Use explicit column names that exist in the provided schema.
# - Do NOT include markdown, comments, or ```sql fences.
# - Output ONLY the SQL query.

# If the question is ambiguous, choose the safest interpretation that produces valid Spark SQL.



# Question:
# {question}
# """

#         response = self.client.chat.completions.create(
#             model="databricks-meta-llama-3-1-8b-instruct",
#             messages=[{"role": "user", "content": prompt}],
#             temperature=0,
#             max_tokens=300,
#         )

#         return response.choices[0].message.content.strip()

#     # -----------------------------
#     # SQL EXECUTION
#     # -----------------------------
#     def run_sql(self, query: str) -> pd.DataFrame:
#         with sql.connect(
#         server_hostname="dbc-741c7540-6155.cloud.databricks.com",
#         http_path="/sql/1.0/warehouses/e2eb5bcf69add002",
#         access_token=os.environ["DATABRICKS_TOKEN"],
#                 ) as connection:
#             return pd.read_sql(query, connection)

#     # -----------------------------
#     # VECTOR SEARCH RETRIEVAL
#     # -----------------------------
#     def retrieve_context(self, question: str):
#         docs = self.ingredient_summary_vs.similarity_search(question, k=3)
#         if not docs:
#             docs = self.pizza_vs.similarity_search(question, k=3)
#         return docs

#     # -----------------------------
#     # MAIN PREDICT
#     # -----------------------------
#     def predict(self, context, model_input):
#         if isinstance(model_input, pd.DataFrame):
#             question = model_input["query"].iloc[0]
#         else:
#             question = model_input["query"]

#         question = str(question)
#         intent = self.detect_query_intent(question)

#         # ---------- SQL ROUTE ----------
#         # if intent == "SQL_ANALYTICS":
#         #     sql = self.generate_sql(question)
#         #     result_df = self.run_sql(sql)

#         #     return [{
#         #                 "answer": result_df.to_json(orient="records"),
#         #                 "intent": "SQL_ANALYTICS",
#         #                 "sql": sql,
#         #                 "num_rows": len(result_df),
#         #                 "num_documents": 0,
#         #             }]
#         if intent == "SQL_ANALYTICS":
#             sql_query = self.generate_sql(question)
#             result_df = self.run_sql(sql_query)
#             if result_df.empty:
#                     return [{
#                     "answer": "No data found for the given question.",
#                     "intent": "SQL_ANALYTICS",
#                     "sql": sql_query,
#                     "num_rows": 0,
#                     "num_documents": 0,
#                             }]
#             rows_json = result_df.to_json(orient="records")

#             # ---------- LLM EXPLANATION ----------
#             explanation_prompt = f"""
# You are a business analyst.

# User question:
# {question}

# SQL query executed:
# {sql_query}

# SQL result (JSON):
# {rows_json}

# Explain the answer clearly and concisely in plain English.
# If numbers are present, compute totals accurately.
# Do NOT mention SQL or JSON in the final answer.
# Do NOT add assumptions
# """

#             explanation = self.client.chat.completions.create(
#                 # model="databricks-meta-llama-3-1-8b-instruct",
#                 model="databricks-gpt-oss-120b",
#                 messages=[
#             {"role": "system", "content": "You explain SQL results accurately."},
#             {"role": "user", "content": explanation_prompt},
#                     ],
#                 temperature=0.1,
#                 max_tokens=200,
#             )

#             return [{
#                 "answer": explanation.choices[0].message.content,  # human-readable
#                 "intent": "SQL_ANALYTICS",
#                 "sql": sql_query,                                  # traceability
#                 "num_rows": len(result_df),
#                 "num_documents": 0,
#                      }]


#         # ---------- RAG ROUTE ----------
#         docs = self.retrieve_context(question)

#         if not docs:
#             return [{
#                 "answer": "No relevant information found.",
#                 "intent": "SEMANTIC",
#                 "num_documents": 0,
#             }]

#         context_text = "\n".join(d.page_content for d in docs)

#         response = self.client.chat.completions.create(
#             model="databricks-gpt-oss-120b",
#             messages=[
#                 {
#                     "role": "system",
#                     "content": "Answer ONLY using provided context."
#                 },
#                 {
#                     "role": "user",
#                     "content": context_text + "\n\n" + question
#                 }
#             ],
#             temperature=0.3,
#             max_tokens=400,
#         )

#         return [{
#                 "answer": response.choices[0].message.content,
#                 "intent": "SEMANTIC",
#                 "sql": "",
#                 "num_rows": 0,
#                 "num_documents": len(docs),
#                 }]


In [0]:
# import os
# import json
# import pandas as pd
# import mlflow.pyfunc
# from openai import OpenAI
# from databricks import sql
# from databricks_langchain import DatabricksVectorSearch


# class PizzaRAGPyFuncModel(mlflow.pyfunc.PythonModel):

#     # ------------------------------------------------------------------
#     # LOAD CONTEXT
#     # ------------------------------------------------------------------
#     def load_context(self, context):

#         self.client = OpenAI(
#             base_url=f"{os.environ['DATABRICKS_HOST']}/serving-endpoints",
#             api_key=os.environ["DATABRICKS_TOKEN"],
#         )

#         self.ingredient_summary_vs = DatabricksVectorSearch(
#             index_name="retail_analytics.pizza.gold_semantic_ingredient_summary_index",
#             endpoint="pizza-rag-endpoint",
#         )

#         self.pizza_vs = DatabricksVectorSearch(
#             index_name="retail_analytics.pizza.gold_semantic_pizza_index",
#             endpoint="pizza-rag-endpoint",
#         )

#     # ------------------------------------------------------------------
#     # STEP 1: QUERY PLANNER (LLM, JSON ONLY)
#     # ------------------------------------------------------------------
    
#     def plan_query(self, question: str) -> dict:
#         prompt = f"""
# Return ONLY valid JSON. No text. No markdown.

# Schema:
# {{
#   "domain": "SALES | INGREDIENT | SEMANTIC",
#   "time_granularity": "DAILY | MONTHLY | NONE",
#   "metric": "REVENUE | SALES | INGREDIENT_USAGE | NONE",
#   "aggregation": "SUM | MAX | MIN | AVG | NONE",
#   "filters": {{
#     "year": null,
#     "month": null,
#     "day": null
#   }}
# }}

# Question:
# {question}
# """

#         try:
#             response = self.client.chat.completions.create(
#                 model="databricks-meta-llama-3-1-8b-instruct",
#                 messages=[{"role": "user", "content": prompt}],
#                 temperature=0,
#                 max_tokens=200,
#             )

#             raw = response.choices[0].message.content.strip()

#         # ---- HARD CLEANING ----
#             raw = raw.replace("```json", "").replace("```", "").strip()

#             plan = json.loads(raw)

#         # ---- SCHEMA GUARD ----
#             required_keys = {"domain", "time_granularity", "metric", "aggregation", "filters"}
#             if not required_keys.issubset(plan):
#                 raise ValueError("Invalid plan schema")

#             return plan

#         except Exception as e:
#         # ---- SAFE FALLBACK (CRITICAL) ----
#             return {
#             "domain": "SEMANTIC",
#             "time_granularity": "NONE",
#             "metric": "NONE",
#             "aggregation": "NONE",
#             "filters": {
#                 "year": None,
#                 "month": None,
#                 "day": None
#                 }
#             }



#     # ------------------------------------------------------------------
#     # STEP 2: ROUTE TABLE & COLUMNS (NO LLM)
#     # ------------------------------------------------------------------
#     def resolve_sql_plan(self, plan: dict) -> dict:

#         if plan["domain"] == "INGREDIENT":
#             return {
#                 "table": "retail_analytics.pizza.gold_ingredient_usage",
#                 "metric_column": "total_ingredient_grams",
#             }

#         if plan["time_granularity"] == "DAILY":
#             return {
#                 "table": "retail_analytics.pizza.gold_pizza_metrics_daily",
#                 "metric_column": "revenue_perday",
#             }

#         if plan["time_granularity"] == "MONTHLY":
#             return {
#                 "table": "retail_analytics.pizza.gold_pizza_metrics_monthly",
#                 "metric_column": "total_revenue"
#                 if plan["metric"] == "REVENUE"
#                 else "total_pizzas_sold",
#             }

#         return {}

#     # ------------------------------------------------------------------
#     # STEP 3: SAFE SQL BUILDER (CODE, NOT LLM)
#     # ------------------------------------------------------------------
#     def build_sql(self, plan: dict, sql_meta: dict) -> str:

#         metric = sql_meta["metric_column"]
#         table = sql_meta["table"]
#         agg = plan["aggregation"]

#         where = []
#         for k, v in plan["filters"].items():
#             if v is not None:
#                 where.append(f"{k} = {v}")

#         where_clause = f"WHERE {' AND '.join(where)}" if where else ""

#         return f"""
# SELECT
#   {agg}({metric}) AS value
# FROM
#   {table}
# {where_clause}
# """.strip()

#     # ------------------------------------------------------------------
#     # STEP 4: SQL EXECUTION
#     # ------------------------------------------------------------------
#     def run_sql(self, query: str) -> pd.DataFrame:
#         with sql.connect(
#             server_hostname="dbc-741c7540-6155.cloud.databricks.com",
#             http_path="/sql/1.0/warehouses/e2eb5bcf69add002",
#             access_token=os.environ["DATABRICKS_TOKEN"],
#         ) as conn:
#             return pd.read_sql(query, conn)

#     # ------------------------------------------------------------------
#     # STEP 5: EXPLAIN RESULTS (LLM)
#     # ------------------------------------------------------------------
#     def explain_result(self, question: str, result_df: pd.DataFrame) -> str:
#         prompt = f"""
# Explain the result in simple business language.

# Question:
# {question}

# Result:
# {result_df.to_dict(orient="records")}

# Do NOT mention SQL.
# """

#         response = self.client.chat.completions.create(
#             model="databricks-gpt-oss-120b",
#             messages=[{"role": "user", "content": prompt}],
#             temperature=0.1,
#             max_tokens=150,
#         )

#         return response.choices[0].message.content

#     # ------------------------------------------------------------------
#     # MAIN PREDICT
#     # ------------------------------------------------------------------
#     def predict(self, context, model_input):

#         question = (
#             model_input["query"].iloc[0]
#             if isinstance(model_input, pd.DataFrame)
#             else model_input["query"]
#         )

#         # 1️⃣ Plan
#         plan = self.plan_query(question)

#         # 2️⃣ Semantic-only route
#         if plan["domain"] == "SEMANTIC":
#             docs = self.ingredient_summary_vs.similarity_search(question, k=3)
#             context_text = "\n".join(d.page_content for d in docs)

#             response = self.client.chat.completions.create(
#                 model="databricks-gpt-oss-120b",
#                 messages=[
#                     {"role": "system", "content": "Answer using context only"},
#                     {"role": "user", "content": context_text + "\n\n" + question},
#                 ],
#                 temperature=0.3,
#                 max_tokens=300,
#             )

#             return [{
#                 "answer": response.choices[0].message.content,
#                 "intent": "SEMANTIC",
#                 "sql": "",
#                 "num_rows": 0,
#                 "num_documents": len(docs),
#             }]

#         # 3️⃣ SQL route
#         sql_meta = self.resolve_sql_plan(plan)
#         sql_query = self.build_sql(plan, sql_meta)
#         df = self.run_sql(sql_query)

#         if df.empty:
#             return [{
#                 "answer": "No data found for the given question.",
#                 "intent": "SQL_ANALYTICS",
#                 "sql": sql_query,
#                 "num_rows": 0,
#                 "num_documents": 0,
#             }]

#         explanation = self.explain_result(question, df)

#         return [{
#             "answer": explanation,
#             "intent": "SQL_ANALYTICS",
#             "sql": sql_query,
#             "num_rows": len(df),
#             "num_documents": 0,
#         }]


In [0]:
import os
import json
import pandas as pd
import mlflow.pyfunc
from openai import OpenAI
from databricks import sql
from databricks_langchain import DatabricksVectorSearch


class PizzaRAGPyFuncModel(mlflow.pyfunc.PythonModel):

    # ==========================================================
    # LOAD CONTEXT
    # ==========================================================
    def load_context(self, context):

        self.client = OpenAI(
            base_url=f"{os.environ['DATABRICKS_HOST']}/serving-endpoints",
            api_key=os.environ["DATABRICKS_TOKEN"],
        )

        self.ingredient_summary_vs = DatabricksVectorSearch(
            index_name="retail_analytics.pizza.gold_semantic_ingredient_summary_index",
            endpoint="pizza-rag-endpoint",
        )

        self.pizza_vs = DatabricksVectorSearch(
            index_name="retail_analytics.pizza.gold_semantic_pizza_index",
            endpoint="pizza-rag-endpoint",
        )

    # ==========================================================
    # STEP 1 — QUERY PLANNER (LLM → STRICT JSON)
    # ==========================================================
    def plan_query(self, question: str) -> dict:

        prompt = f"""
Return ONLY valid JSON. No markdown. No explanation.

Schema:
{{
  "domain": "SALES | INGREDIENT | SEMANTIC",
  "time_granularity": "DAILY | MONTHLY | NONE",
  "metric": "REVENUE | SALES | INGREDIENT_USAGE | NONE",
  "aggregation": "SUM | MAX | MIN | AVG | NONE",
  "filters": {{
    "year": null,
    "month": null,
    "day": null
  }}
}}

Question:
{question}
"""

        try:
            response = self.client.chat.completions.create(
                model="databricks-meta-llama-3-1-8b-instruct",
                messages=[{"role": "user", "content": prompt}],
                temperature=0,
                max_tokens=200,
            )

            raw = response.choices[0].message.content.strip()
            raw = raw.replace("```json", "").replace("```", "").strip()
            plan = json.loads(raw)

            required = {"domain", "time_granularity", "metric", "aggregation", "filters"}
            if not required.issubset(plan):
                raise ValueError("Invalid planner output")

            return plan

        except Exception:
            # HARD FAILSAFE
            return {
                "domain": "SEMANTIC",
                "time_granularity": "NONE",
                "metric": "NONE",
                "aggregation": "NONE",
                "filters": {"year": None, "month": None, "day": None},
            }

    # ==========================================================
    # STEP 2 — ENTITY QUESTION DETECTOR (CODE)
    # ==========================================================
    # def is_entity_question(self, question: str) -> bool:
    #     q = question.lower()
    #     return any(k in q for k in ["which", "what", "most", "highest", "top","least","lowest"])
    def parse_entity_intent(self, question: str) -> dict:
        q = question.lower()

        return {
            "is_entity": any(k in q for k in ["which", "what", "top", "most", "highest", "lowest"]),
            "ranking": (
                "MAX" if any(k in q for k in ["most", "highest", "max"]) else
                "MIN" if any(k in q for k in ["least", "lowest", "min"]) else
                None
            ),
            "top_k": (
                int(next((w for w in q.split() if w.isdigit()), "1"))
            ),
            "needs_window": any(k in q for k in [
                "per month", "per year", "each month", "each year", "trend", "rank"
            ])
        }


    # def needs_join(self, question: str) -> bool:
    # q = question.lower()
    # return (
    #     "ingredient" in q
    #     and any(k in q for k in ["revenue", "sales"])
    # )
    def resolve_joins(self, plan: dict) -> dict:
        """
        Decides base table and required joins.
        """

        joins = []

        # -------------------------------
        # BASE TABLE
        # -------------------------------
        if plan["domain"] == "INGREDIENT":
            base_table = "retail_analytics.pizza.gold_ingredient_usage"
            base_alias = "i"
        elif plan["time_granularity"] == "DAILY":
            base_table = "retail_analytics.pizza.gold_pizza_metrics_daily"
            base_alias = "d"
        else:
            base_table = "retail_analytics.pizza.gold_pizza_metrics_monthly"
            base_alias = "m"

        # -------------------------------
        # INGREDIENT → SALES/REVENUE
        # -------------------------------
        if plan["domain"] == "INGREDIENT" and plan["metric"] in ["REVENUE", "SALES"]:
            joins.append({
                "table": "retail_analytics.pizza.gold_pizza_metrics_monthly",
                "alias": "m",
                "on": "i.pizza_name_id = m.pizza_name_id AND i.month = m.month AND i.year = m.year"
            })

        # -------------------------------
        # DAILY → MONTHLY (rollups)
        # -------------------------------
        if plan["time_granularity"] == "DAILY" and plan["metric"] == "REVENUE":
            joins.append({
                "table": "retail_analytics.pizza.gold_pizza_metrics_monthly",
                "alias": "m",
                "on": "d.pizza_name_id = m.pizza_name_id AND d.month = m.month AND d.year = m.year"
            })

        # -------------------------------
        # INGREDIENT + DAILY + MONTHLY
        # -------------------------------
        if (
            plan["domain"] == "INGREDIENT"
            and plan["time_granularity"] == "DAILY"
            and plan["metric"] == "REVENUE"
        ):
            joins.append({
                "table": "retail_analytics.pizza.gold_pizza_metrics_daily",
                "alias": "d",
                "on": "i.pizza_name_id = d.pizza_name_id AND i.day = d.day AND i.month = d.month AND i.year = d.year"
            })
            joins.append({
                "table": "retail_analytics.pizza.gold_pizza_metrics_monthly",
                "alias": "m",
                "on": "d.pizza_name_id = m.pizza_name_id AND d.month = m.month AND d.year = m.year"
            })

        return {
            "base_table": base_table,
            "base_alias": base_alias,
            "joins": joins
        }



    # ==========================================================
    # STEP 3 — SQL PLAN RESOLUTION (NO LLM)
    # ==========================================================
    def resolve_sql_plan(self, plan: dict) -> dict:

        if plan["domain"] == "INGREDIENT":
            return {
                "table": "retail_analytics.pizza.gold_ingredient_usage",
                "metric_column": "total_ingredient_grams",
                "entity_column": "ingredient",
            }

        if plan["time_granularity"] == "DAILY":
            return {
                "table": "retail_analytics.pizza.gold_pizza_metrics_daily",
                "metric_column": "revenue_perday",
                "entity_column": "pizza_name_id",
            }

        if plan["time_granularity"] == "MONTHLY":
            return {
                "table": "retail_analytics.pizza.gold_pizza_metrics_monthly",
                "metric_column": "total_revenue"
                if plan["metric"] == "REVENUE"
                else "total_pizzas_sold",
                "entity_column": "pizza_name_id",
            }

        return {}

    # ==========================================================
    # STEP 4 — SAFE SQL BUILDER (DETERMINISTIC)
    # ==========================================================

    def build_sql(self, plan: dict, meta: dict, question: str) -> str:

        entity_meta = self.parse_entity_intent(question)
        join_plan = self.resolve_joins(plan)

        base = join_plan["base_table"]
        b = join_plan["base_alias"]
        joins = join_plan["joins"]

        metric = meta["metric_column"]
        entity = meta.get("entity_column")
        agg = plan["aggregation"]

        # --------------------------------
        # PARTITION LOGIC (WINDOW)
        # --------------------------------
        partition_cols = []
        if plan["time_granularity"] == "MONTHLY":
            partition_cols = ["year", "month"]
        elif plan["time_granularity"] == "DAILY":
            partition_cols = ["year", "month", "day"]

        partition_sql = ", ".join(f"{b}.{c}" for c in partition_cols)

        # --------------------------------
        # WHERE
        # --------------------------------
        filters = []
        for k, v in plan["filters"].items():
            if v is not None:
                filters.append(f"{b}.{k} = {v}")

        where_clause = f"WHERE {' AND '.join(filters)}" if filters else ""

        # --------------------------------
        # JOIN CLAUSE
        # --------------------------------
        join_sql = "\n".join(
            f"JOIN {j['table']} {j['alias']} ON {j['on']}"
            for j in joins
        )

        # --------------------------------
        # WINDOW FUNCTION QUERY
        # --------------------------------
        if entity_meta["needs_window"] and entity:

            order_dir = "DESC" if entity_meta["ranking"] != "MIN" else "ASC"
            limit_k = entity_meta["top_k"]

            return f"""
WITH ranked_data AS (
  SELECT
    {b}.{entity},
    {partition_sql},
    {agg}({metric}) AS value,
    RANK() OVER (
      PARTITION BY {partition_sql}
      ORDER BY {agg}({metric}) {order_dir}
    ) AS rnk
  FROM {base} {b}
  {join_sql}
  {where_clause}
  GROUP BY {b}.{entity}, {partition_sql}
)
SELECT *
FROM ranked_data
WHERE rnk <= {limit_k}
ORDER BY {partition_sql}, rnk
""".strip()

    # --------------------------------
    # ENTITY (NO WINDOW)
    # --------------------------------
        if entity_meta["is_entity"] and entity:
            order_dir = "DESC" if entity_meta["ranking"] != "MIN" else "ASC"
            limit_k = entity_meta["top_k"]

            return f"""
SELECT
  {b}.{entity},
  {agg}({metric}) AS value
FROM {base} {b}
{join_sql}
{where_clause}
GROUP BY {b}.{entity}
ORDER BY value {order_dir}
LIMIT {limit_k}
""".strip()

    # --------------------------------
    # METRIC ONLY
    # --------------------------------
        return f"""
SELECT
  {agg}({metric}) AS value
FROM {base} {b}
{join_sql}
{where_clause}
""".strip()


    # ==========================================================
    # STEP 5 — SQL EXECUTION
    # ==========================================================
    def run_sql(self, query: str) -> pd.DataFrame:
        with sql.connect(
            server_hostname="dbc-741c7540-6155.cloud.databricks.com",
            http_path="/sql/1.0/warehouses/e2eb5bcf69add002",
            access_token=os.environ["DATABRICKS_TOKEN"],
        ) as conn:
            return pd.read_sql(query, conn)

    # ==========================================================
    # STEP 6 — RESULT EXPLANATION (LLM)
    # ==========================================================
    def explain_result(self, question: str, df: pd.DataFrame) -> str:

        prompt = f"""
Explain the result in clear business language.

Question:
{question}

Result:
{df.to_dict(orient="records")}

Rules:
- Do NOT mention SQL
- Do NOT mention JSON
- Do NOT add assumptions
"""

        response = self.client.chat.completions.create(
            model="databricks-gpt-oss-120b",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            max_tokens=150,
        )

        return response.choices[0].message.content

    # ==========================================================
    # MAIN PREDICT
    # ==========================================================
    def predict(self, context, model_input):

        question = (
            model_input["query"].iloc[0]
            if isinstance(model_input, pd.DataFrame)
            else model_input["query"]
        )

        # 1️⃣ PLAN
        plan = self.plan_query(question)

        # 2️⃣ SEMANTIC ROUTE
        if plan["domain"] == "SEMANTIC":
            docs = self.ingredient_summary_vs.similarity_search(question, k=3)
            context_text = "\n".join(d.page_content for d in docs)

            response = self.client.chat.completions.create(
                model="databricks-gpt-oss-120b",
                messages=[
                    {"role": "system", "content": "Answer using provided context only"},
                    {"role": "user", "content": context_text + "\n\n" + question},
                ],
                temperature=0.3,
                max_tokens=300,
            )

            return [{
                "answer": response.choices[0].message.content,
                "intent": "SEMANTIC",
                "sql": "",
                "num_rows": 0,
                "num_documents": len(docs),
            }]

        # 3️⃣ SQL ROUTE
        meta = self.resolve_sql_plan(plan)
        sql_query = self.build_sql(plan, meta, question)
        df = self.run_sql(sql_query)

        if df.empty:
            return [{
                "answer": "No data found for the given question.",
                "intent": "SQL_ANALYTICS",
                "sql": sql_query,
                "num_rows": 0,
                "num_documents": 0,
            }]

        explanation = self.explain_result(question, df)

        return [{
            "answer": explanation,
            "intent": "SQL_ANALYTICS",
            "sql": sql_query,
            "num_rows": len(df),
            "num_documents": 0,
        }]


In [0]:
import mlflow
from mlflow.models import ModelSignature
from mlflow.types import Schema, ColSpec

signature = ModelSignature(
    inputs=Schema([
        ColSpec("string", "query")
    ]),
    outputs=Schema([
        ColSpec("string", "answer"),
        ColSpec("string", "intent"),
        ColSpec("string", "sql"),
        ColSpec("long", "num_rows"),
        ColSpec("long", "num_documents"),
    ]),
)

with mlflow.start_run():
    mlflow.pyfunc.log_model(
        name="pizza_rag_model",
        python_model=PizzaRAGPyFuncModel(),
        signature=signature,
        input_example={"query": "how much revenue generated in 25 december and year 2015"},
        registered_model_name="retail_analytics.pizza.pizza_rag_models",
    )
